In [8]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.getcwd(), ".."))

from Models.splinedist import SplineDist
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor, RichProgressBar, RichModelSummary
from Datasets.DSB18 import Nuclie_datamodule 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:

logger = pl_loggers.TensorBoardLogger(
    save_dir="pl_logs",
    name="spline_dist",
    log_graph=True,
    default_hp_metric=True)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="Models/weights",
    filename="splineDist-{epoch:02d}-{val_loss:.2f}",
    save_top_k=5,
    mode="min",
)

earlystoping_callback = EarlyStopping("val_acc")

lr_logger = LearningRateMonitor(logging_interval="step")
summary = RichModelSummary(max_depth=-1)
progressbar = RichProgressBar()

callbacks = [progressbar, checkpoint_callback,
             lr_logger]


In [ ]:
model = SplineDist()
trainer = pl.Trainer(max_epochs=1,
                     gpus=1,
                     callbacks=callbacks,
                     logger=logger,
                     profiler="simple")
trainer.fit(model, Nuclie_datamodule())


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃   ┃ Name                           ┃ Type       ┃ Params ┃       In sizes ┃     Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 0 │ segmentationBackbone           │ UNet       │ 17.3 M │    [1, 3, 256, │ [1, 128, 254, │
│   │                                │            │        │           256] │          254] │
│ 1 │ objectFinder                   │ Sequential │  1.2 K │  [1, 128, 254, │   [1, 1, 254, │
│   │                                │            │        │           254] │          254] │
│ 2 │ ControlPointsAngleRegressor    │ Conv2d     │  9.2 K │  [1, 128, 254, │   [1, 8, 254, │
│   │                                │            │        │           254] │          254] │
│ 3 │ ControlPointsDistanceRegressor │ Conv2d     │  9.2 K │  [1, 128, 254, │   [1, 8, 254, │
│   │                                │            │        │           254] │          254] │
└───┴────────────────────────────────┴────────────┴────────┴────────────────┴───────────────┘

Trainable params: 17.4 M                                                                     
Non-trainable params: 0                                                                      
Total params: 17.4 M                                                                         
Total estimated model params size (MB): 69

Output()